In [3]:
!pip install tensorflow
import boto3
import pickle
from sagemaker import get_execution_role
role = get_execution_role()
bucket = 'saturn-enceladus'
s3client = boto3.client('s3')
response = s3client.get_object(Bucket=bucket, Key='mean_reversion_training_data.pkl')
body = response['Body']

data = pickle.loads(body.read())


Keyring is skipped due to an exception: 'keyring.backends'


In [12]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
def preprocess_v1(data):
    X=[]
    Y=[]
    for datapoint in data:
        scaler = MinMaxScaler()
        action = datapoint[0][0]
        sequence = datapoint[0][1]
        sequence=scaler.fit_transform(sequence)
        outcome = int(datapoint[1])
        action_array = np.full((1,10), action)
        # print(action_array)
        # break
        transposed_sequence = sequence.T
        # final_datapoint = np.append(transposed_sequence, action_array, axis=0).T
        X.append(sequence)
        Y.append(outcome)
        
    return np.array(X), np.array(Y)

x, y = preprocess_v1(data)
print(np.sum(y)/len(y))
X_train, X_test_val, y_train, y_test_val = train_test_split(x, y, test_size=0.5, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_test_val, y_test_val, test_size=0.5, random_state=42)
pickle.dump([(X_train, y_train), (X_test, y_test), (X_val, y_val)], open('train_test_val_split.pkl', 'wb'))

0.7137129109863672
